In [81]:
using DrWatson
using Plots
using IJulia
using SpikingNeuralNetworks
SNN.@load_units;
using SNNUtils

## Sparse matrix representation: 

The sparse matrix representation uses the Julia native SparseArrays package. You can find further info in the [Julia documentation](https://docs.julialang.org/en/v1/stdlib/SparseArrays/#man-csc).

Because network are expressed with matrices, we use the Compressed Sparse Column __(CSC)__ Matrix Storage format.
The internal representation is as follows:

```Julia
struct SparseMatrixCSC{Tv,Ti<:Integer} <: AbstractSparseMatrixCSC{Tv,Ti}
    m::Int                  # Number of rows
    n::Int                  # Number of columns
    colptr::Vector{Ti}      # Column j is in colptr[j]:(colptr[j+1]-1)
    rowval::Vector{Ti}      # Row indices of stored values
    nzval::Vector{Tv}       # Stored values, typically nonzeros
end
```

This representation is sufficient to make any type of operation that we can think of on the matrix. This tutorial will give some snippets of code to be reused to this scope.

### Defining a Sparse Matrix with CSC

To define a sparse matrix we can follow several approaches:
1. A random matrix with random zeros defined by the probability `p`
2. A pre-defined matrix with zeros, and thus impose a sparse representation.
3. A zero sparse matrix and thus fill the elements we need.

In [82]:
using SparseArrays
m = 2 # number of rows
n = 3 # number of columns

# case 1:
p = 0.33
A1 = sprand(m,n,p)

# case 2:
A2 = rand([0,0,1],m,n)
A2 = sparse(A2)

#case 3:
A3 = sprand(m,n,0.)
A3[2,2] = 1
A3[1,3] = 1
A3[1,2] = 1

;

### Accessing Sparse Matrices

To access the sparse matrix we can use the classical matrix access 
```Julia
A1 = sprand(2,3,0.1)
x = A1[1,2] ...
```
However, most of time we want to access only the non-zero values. This can be easily done, but require so ad-hoc code. 

Let's define a matrix `A` and access it:

In [83]:
A = sprand(10,10,0.0)
A[1,2] = 1
A[3,5] = 2
A[8,5] = 3
A[3,10] =4

A

10×10 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   2.0   ⋅    ⋅    ⋅    ⋅   4.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   3.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 

The matrix will be accessed in the column order, left-to-right, because this is the storage format of the CSC:

It follows the algorithm:

1. The loop goes on each index of colptr `i`
2. It creates a range between the `colptr[i]` and  `colptr[i+1]`. This range contains all the non-zero indices in the `i` column.
3. It loops through the indices contained in the `i` column with the temporary variable `st`.
4. It access the non-zero values corresponding to the `st` index: (`nzval[st]`) and the row index of that value (`rowval[st]`).
5. Finally, we have the value for the colum `i`, the value for the row `rowval[st]` and the value in the matrix `nzval[st]`

In [84]:
@unpack colptr, nzval, rowval = A
for i = 1:(length(colptr)-1) # column indices
    for st in colptr[i]:(colptr[i+1]-1) # 
        println("col: $i, st: $st, row: $(rowval[st]) ")
        # println("pre->post: $i -> $(rowval[st]) ")
        println("value => $(nzval[st])")
    end
end

col: 2, st: 1, row: 1 
value => 1.0
col: 5, st: 2, row: 3 
value => 2.0
col: 5, st: 3, row: 8 
value => 3.0
col: 10, st: 4, row: 3 
value => 4.0


### Using Sparse Matrices in SNN

In the example above, we have looped through the columns and found the associated non-zero values. However, sometimes we want to achieve the same operation through the rows. 

The package SNN defines some convenience functions to use the sparse matrix. Using the function `SNN.dsparse` you can get an additional set of iterators that allow transversing the matrix both through the columns and through the rows:

```Julia
rowptr, colptr, I, J, index, V = dsparse(W)
```

Where `I` is equivalent to `rowvals` and `V` contains all the non-zero values.

The iterators for the rows are:
```Julia
    rowptr # pointer to the rows
    J  # value of the colum at the index 
    index # index of the j element of the row
```

Because the values in `J` and `V` are disposed with respect to the CSC format, now we need the  `index` array to access the correct column and value


In [85]:
A = sprand(10,10,0.0)
A[1,5] = 1
A[2,2] = 2
A[2,8] = 3
A[8,10] = 4

rowptr, colptr, I, J, index, V = SNN.dsparse(A)
for i = 1:(length(rowptr)-1) # postsynaptic indices i
    for st = rowptr[i]:(rowptr[i+1]-1) ## 
        println("row: $i, st: $st, col: $(J[index[st]]) ")
        # println("pre->post: $i -> $(rowval[st]) ")
        println("value => $(V[index[st]])")
    end
end

A

row: 1, st: 1, col: 5 
value => 1.0
row: 2, st: 2, col: 2 
value => 2.0
row: 2, st: 3, col: 8 
value => 3.0
row: 8, st: 4, col: 10 
value => 4.0


10×10 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   2.0   ⋅    ⋅    ⋅    ⋅    ⋅   3.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   4.0
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 

### Examples:

#### Connectivity matrix
Our matrix defines the connectivity between N pre-synaptic neurons and M post-synaptic neurons with the matrix W with dimensions M x N.

__Select by pre-synaptic activity__
Upon the spike of the pre-synaptic `i`, we update the synaptic conductance `G` of the post-synaptic cells connected to `i`.


In [86]:

N = 10 #presynaptic
M = 5 # postynaptic
W = sparse(rand([0,1],M,N))

i = 5
## Assign the matrix W the value m at the row m, col i 
for m = 1:M
    W[m,i] = m
end

G= zeros(M)
firePre = falses(N)
firePre[i] = true

@unpack colptr, nzval, rowval = W
for i = 1:(length(colptr)-1) # column indices
    if firePre[i]
        for st in colptr[i]:(colptr[i+1]-1) # 
            G[rowval[st]] += nzval[st]
        end
    end
end

println(W[:,5]) # the values
println(G)
W

  [1]  =  1
  [2]  =  2
  [3]  =  3
  [4]  =  4
  [5]  =  5
[1.0, 2.0, 3.0, 4.0, 5.0]


5×10 SparseMatrixCSC{Int64, Int64} with 28 stored entries:
 1  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  1  ⋅
 1  ⋅  ⋅  ⋅  2  1  1  1  1  1
 1  1  ⋅  ⋅  3  ⋅  ⋅  ⋅  1  ⋅
 1  1  1  ⋅  4  1  ⋅  ⋅  ⋅  ⋅
 1  1  1  1  5  ⋅  1  1  ⋅  1

__Select by post-synaptic activity__
Upon the spike of the post-synaptic `j`, we update the elegibility traces `P` of the pre-synaptic cells connected to `j`.

In [92]:
N = 10 #presynaptic
M = 5 # postynaptic
W = sparse(rand([0,1],M,N))
j = 2
## Assign the matrix W the value n at the col n, row  j 
for n = 1:N
    W[j,n] = n
end

P= zeros(N)
firePost = falses(M)
firePost[j] = true

rowptr, colptr, I, J, index, V = SNN.dsparse(W)
for j = 1:(length(rowptr)-1) # column indices
    if firePost[j]
        for st in rowptr[j]:(rowptr[j+1]-1) # 
            P[J[index[st]]] = V[index[st]]
        end
    end
end

println(W[j, :]) # the values
println(P)
W


  [1 ]  =  1
  [2 ]  =  2
  [3 ]  =  3
  [4 ]  =  4
  [5 ]  =  5
  [6 ]  =  6
  [7 ]  =  7
  [8 ]  =  8
  [9 ]  =  9
  [10]  =  10
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]


5×10 SparseMatrixCSC{Int64, Int64} with 29 stored entries:
 1  1  ⋅  ⋅  ⋅  1  ⋅  1  ⋅   1
 1  2  3  4  5  6  7  8  9  10
 1  ⋅  ⋅  1  ⋅  ⋅  1  1  ⋅   1
 ⋅  ⋅  ⋅  1  1  1  1  ⋅  1   1
 ⋅  ⋅  ⋅  ⋅  1  1  1  ⋅  ⋅   ⋅

In [91]:
W

5×10 SparseMatrixCSC{Int64, Int64} with 34 stored entries:
 ⋅  1  ⋅  ⋅  1  1  ⋅  ⋅  1   1
 1  2  3  4  5  6  7  8  9  10
 1  1  1  1  1  ⋅  1  1  ⋅   ⋅
 1  1  1  1  ⋅  ⋅  1  1  1   ⋅
 1  ⋅  1  ⋅  1  ⋅  ⋅  1  ⋅   1

In [93]:
rowptr

6-element Vector{Int64}:
  1
  6
 16
 21
 27
 30